I have a bachelor's in finance and a master's in IT. I would like to combine my skills with my interest in data analysis. Thus, I decided to do a side project researching financial markets.

# Data Preparation
I did not find any convenient API to collect financial data in Taiwan. Therefore, I decided to built web crawlers
## Data Collection

##### import python packages

In [27]:
import requests
import pandas as pd
import numpy as np
import datetime
import time
from io import StringIO

### Financial Statement Crawler
Income Statement | Balance Sheet | Financial Ratios
#### Statement of one period

In [2]:
def income_statement(year, season):
    
    # Transform the AD year input to National year system
    if year >= 1000:
        year -= 1911
    
    # starting from 102 national year (2013 AD year), IFRSs system is applied
    # before then, was GAAP
    if year >= 102:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif year < 102:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t51sb13'
    else:
        print('type does not match')

        
    r = requests.post(url, 
                      {'encodeURIComponent':1,
                       'step':1,
                       'firstin':1,
                       'off':1,
                       'TYPEK':'sii',
                       'year':str(year).zfill(3),
                       'season':str(season).zfill(2)})
    r.encoding = 'utf8'
    
    # receive a list of DataFrames
    DFs = pd.read_html(r.text)

    # data with GAAP system sometimes returns duplicate columns 
    if year < 102:
        for i in DFs:
            try:
                i.columns = i.columns.droplevel()
            except:
                pass
    
    # the first element is some notifications
    return DFs[1:]


In [3]:
def balance_sheet(year, season):
    
    # Transform the AD year input to National year system
    if year >= 1000:
        year -= 1911
    
    # starting from 102 national year (2013 AD year), IFRSs system is applied
    # before then, was GAAP
    if year >= 102:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif year < 102:
        url = 'https://mops.twse.com.tw/mops/web/ajax_t51sb12'
    else:
        print('type does not match')


    r = requests.post(url, 
                      {'encodeURIComponent':1,
                       'step':1,
                       'firstin':1,
                       'off':1,
                       'TYPEK':'sii',
                       'year':str(year).zfill(3),
                       'season':str(season).zfill(2)})
    r.encoding = 'utf8'
    
    # receive a list of DataFrames
    DFs = pd.read_html(r.text)

    
    if year >= 102:
        # the first element is some notifications
        DFs = DFs[1:]
    elif year < 102:
        # there are some notes between each table
        DFs = DFs[1::2]
    return DFs


In [4]:
def financial_rate(year):

    url = "https://mops.twse.com.tw/mops/web/ajax_t51sb02"

    # Transform the AD year input to National year system
    if year >= 1000:
        year -= 1911

    # starting from 102 national year (2013 AD year), IFRSs system is applied
    # before then, was GAAP
    if year>=102:
        r = requests.post(url, {
            'encodeURIComponent':1,
            'step':1,
            'run':"Y",
            'firstin':1,
            'off':1,
            'TYPEK':'sii',
            'year':str(year).zfill(3),
            'ifrs':"Y",
            })
    elif year<102:
        r = requests.post(url, {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':'sii',
            'year':str(year).zfill(3),
            })
    r.encoding = 'utf8'
    
    # receive a list of DataFrames
    DFs = pd.read_html(r.text)
    
    # the first element is some notifications
    return DFs[1:]


#### Statement for range of periods

In [5]:
def financial_deadline(data, year, season, DFs):
    """
    data are split into different tables based on industries
    deadlines for releasing finanical statement for financial industry are different 
    from other industries.
    """
    if year>=2013:
        if season==1:
            data[datetime.date(year, 5, 30)] = DFs.pop(-3)
            data[datetime.date(year, 5, 15)] = pd.concat(DFs)
        elif season == 2:
            data[datetime.date(year, 8, 14)] = pd.concat([DFs.pop(2),DFs.pop(-1)])
            data[datetime.date(year, 8, 31)] = pd.concat(DFs)
        elif season == 3:
            data[datetime.date(year, 11, 29)] = DFs.pop(-3)
            data[datetime.date(year, 11, 14)] = pd.concat(DFs)
        elif season == 4:
            data[datetime.date(year+1, 3, 31)] = pd.concat(DFs)
    elif year<2013:
        if season==1:
            data[datetime.date(year, 5, 15)] = pd.concat(DFs)
        elif season == 2:
            data[datetime.date(year, 8, 31)] = dfs.pop(-4)
            data[datetime.date(year, 9, 13)] = pd.concat(DFs)
        elif season == 3:
            data[datetime.date(year, 11, 14)] = pd.concat(DFs)
        elif season == 4:
            data[datetime.date(year+1, 3, 31)] = pd.concat(DFs)
    return data

In [26]:
def statementCrawler(end_year,end_quater, start_year, statement_type, allow_continuous_fail_count=1):

    # init variables
    data = {}
    fail_count = 0
    count_period = 0

    n_years = int(start_year)
    year = int(end_year)
    season = int(end_quater)

    while year >= n_years:
        print('parsing', str(year)+str(season).zfill(2))
        
        try:
            if statement_type == "balance_sheet":
                dfs = balance_sheet(year,season)
                data = financial_deadline(data, year, season, dfs)
            elif statement_type == "income statement":
                dfs = income_statement(year,season)
                data = financial_deadline(data, year, season, dfs)
            elif statement_type == "financial_rate":
                dfs = pd.concat(financial_rate(year))
                dfs["財報日期"] = str(year)+"04"
                data[datetime.date(year+1, 3, 31)] = dfs
                season = 1
            else:
                print("invalid statement input")
                break
            
            print('success!')
            count_period +=1
            print(count_period)
            fail_count = 0
            
            # only continues if success
            if season ==1:
                year-=1
                season = 4
            else:
                season -=1

        except:
            print('Failed')
            fail_count += 1
            if fail_count == allow_continuous_fail_count:
                raise
                break

        time.sleep(10)
    return data

### Stock Trade Price Crawler

In [28]:
def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret


In [31]:
def stockpriceCrawler(startDate, endDate=datetime.date.today()):
    
    # init variables
    data = {}
    fail_count = 0
    count_day = 0
    
    # avoiding long holidays
    allow_continuous_fail_count = 25
    
    startDate = pd.to_datetime(str(startDate)).date()
    
    while startDate <= endDate:

        print('parsing', endDate)
        
        try:
            data[endDate] = crawl_price(endDate)
            print('success!')
            count_day +=1
            print(count_day)
            fail_count = 0
        except:
            print('fail! check the date is holiday')
            fail_count += 1
            if fail_count == allow_continuous_fail_count:
                raise
                break

        # backward one day
        endDate -= datetime.timedelta(days=1)
        time.sleep(15)
        
    return pd.concat(data).rename_axis(["日期","證券代號"]).iloc[:,:-1]

## Data Cleansing
### Financial statement 

In [ ]:
# fill up missing data with Nan
BS_DF = pd.concat(BS_Dict).applymap(lambda x: x if x != '--' else np.nan)
IS_DF = pd.concat(IS_Dict).applymap(lambda x: x if x != '--' else np.nan)
FR_DF = pd.concat(FR_Dict).applymap(lambda x: x if x != '--' else np.nan)

# remove non valid data
BS_DF = BS_DF[BS_DF['公司代號'] != '公司代號']
IS_DF = IS_DF[IS_DF['公司代號'] != '公司代號']
FR_DF = FR_DF[FR_DF[('公司代號','公司代號')] != '公司代號']

BS_DF = BS_DF[~BS_DF['公司代號'].isnull()]
IS_DF = IS_DF[~IS_DF['公司代號'].isnull()]
FR_DF = FR_DF[~FR_DF[('公司代號','公司代號')].isnull()]

# remove duplicate columns and reset index name
BS_DF = BS_DF.rename_axis(["日期","沒用"]).reset_index().set_index(["日期","公司名稱"]).iloc[:,1:]
IS_DF = IS_DF.rename_axis(["日期","沒用"]).reset_index().set_index(["日期","公司名稱"]).iloc[:,1:]
FR_DF = FR_DF.rename_axis(["日期","沒用"]).reset_index().set_index(["日期",('公司代號','公司代號')]).iloc[:,1:]
FR_DF.columns = FR_DF.columns.droplevel(0)
FR_Col = FR_DF.columns.to_list()
FR_Col[-1] = "財報日期"
FR_DF.columns = FR_Col
FR_DF.rename_axis(mapper=["日期","公司代號"],inplace = True)

In [ ]:
# inconsistent column names for Total Asset
# checked the sum data amount of all different  Total Asset which matched the total data amount
len(BS_DF["資產合計"][~BS_DF["資產合計"].isna()])+len(BS_DF["資產總計"][~BS_DF["資產總計"].isna()])+len(BS_DF["資產總額"][~BS_DF["資產總額"].isna()])
# combined as a new column
BS_DF["總資產"] = BS_DF["資產合計"].fillna(0.0)+BS_DF["資產總計"].fillna(0.0)+BS_DF["資產總額"].fillna(0.0)

# same for the Total Liabilities
len(BS_DF["負債合計"][~BS_DF["負債合計"].isna()])+len(BS_DF["負債總計"][~BS_DF["負債總計"].isna()])+len(BS_DF["負債總額"][~BS_DF["負債總額"].isna()])
BS_DF["總負債"] = BS_DF["負債合計"].fillna(0.0)+BS_DF["負債總計"].fillna(0.0)+BS_DF["負債總額"].fillna(0.0)

# same for the Total Equities
len(BS_DF["股東權益合計"][~BS_DF["股東權益合計"].isna()])+len(BS_DF["股東權益總計"][~BS_DF["股東權益總計"].isna()])+len(BS_DF["權益合計"][~BS_DF["權益合計"].isna()])+len(BS_DF["權益總額"][~BS_DF["權益總額"].isna()])+len(BS_DF["權益總計"][~BS_DF["權益總計"].isna()])
BS_DF["總權益"] = BS_DF["股東權益合計"].fillna(0.0)+BS_DF["股東權益總計"].fillna(0.0)+BS_DF["權益合計"].fillna(0.0)+BS_DF["權益總額"].fillna(0.0)+BS_DF["權益總計"].fillna(0.0) 


In [141]:
bid = BS_DF.index
iid = IS_DF.index
bid.difference(iid)

# have checked the data are test company id 

MultiIndex(levels=[[200802, 201104], [3990, 910482]],
           codes=[[0, 1], [1, 0]],
           names=['財報日期', '公司代號'],
           sortorder=0)

In [ ]:
FinState_DF = BS_DF.merge(IS_DF,how = "outer",on = ["財報日期","公司代號","日期","公司名稱"])
FinState_DF = FinState_DF.merge(FR_DF,how="outer",on=["財報日期","公司代號"])

### Combining Financial Statement with Stock Price

In [ ]:
stock_df = pd.read_csv("TWstock_070118_190425.csv",usecols = ["日期","證券代號","收盤價","本益比"])

# preparing columns for stock prices on buy and sell date
FinState_DF["日期"] = pd.to_datetime(FinState_DF["日期"])

# day before next statement release
FinState_DF["下次財報前一天"] = np.nan
for i in FinState_DF["公司名稱"].unique():
    FinState_DF["下次財報前一天"][FinState_DF["公司名稱"]==i] = (FinState_DF["日期"][FinState_DF["公司名稱"]==i]-dt.timedelta(1)).shift(-1)

In [ ]:
FinState_DF["公司代號"] = FinState_DF["公司代號"].apply(str)
stock_df["日期"] = pd.to_datetime(stock_df["日期"])

# statement release date as buy date
# day before next statement release as sell date
FinState_DF["買入日期"] = pd.to_datetime(FinState_DF["日期"])
FinState_DF["賣出日期"] = pd.to_datetime(FinState_DF["下次財報前一天"])

stock_df["公司代號"] = stock_df["證券代號"]
stock_df.drop("證券代號",axis=1,inplace=True)
stock_df.set_index(["日期","公司代號"],inplace=True)

# merge finanical statement with stock price
# if buy date has no price data, check next day
FinState_DF = FinState_DF.merge(stock_df,left_on=["買入日期","公司代號"],right_on=["日期","公司代號"],how="left")
while len(FinState_DF["買入日期"][FinState_DF["收盤價"].isna()])>0:
    FinState_DF["買入日期"] = pd.concat([pd.to_datetime(FinState_DF["買入日期"][FinState_DF["收盤價"].isna()])+dt.timedelta(1),pd.to_datetime(FinState_DF["買入日期"][~FinState_DF["收盤價"].isna()])]).sort_index()
    FinState_DF = FinState_DF.drop(["收盤價","本益比"],axis=1)
    FinState_DF = FinState_DF.merge(stock_df,left_on=["買入日期","公司代號"],right_on=["日期","公司代號"],how="left")

# merge finanical statement with stock price
# if sell date has no price data, check previous day
FinState_DF = FinState_DF.merge(stock_df["收盤價"],left_on=["賣出日期","公司代號"],right_on=["日期","公司代號"],how="left",suffixes=("","_賣出"))
while len(FinState_DF["賣出日期"][FinState_DF["收盤價_賣出"].isna()])>0:
    FinState_DF["賣出日期"] = pd.concat([pd.to_datetime(FinState_DF["賣出日期"][FinState_DF["收盤價_賣出"].isna()])-dt.timedelta(1),pd.to_datetime(FinState_DF["賣出日期"][~FinState_DF["收盤價_賣出"].isna()])]).sort_index()
    FinState_DF = FinState_DF.drop("收盤價_賣出",axis=1)
    FinState_DF = FinState_DF.merge(stock_df["收盤價"],left_on=["賣出日期","公司代號"],right_on=["日期","公司代號"],how="left",suffixes=("","_賣出"))

In [ ]:
FinState_DF.to_csv("FinancialStatement_with_StockPrice_0801_1804.csv")

Future work:

I planned to research on finding the relations between finanical ratios and stock price growth.

There are still quite a few cleaning work yet to be done.

However, as I found a nice API for getting US financial data. I decided to move on to that.
